### Season 2021-2022 

# Import Libraries

In [89]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Initialization site

In [90]:
standings_url = "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats"

In [91]:
data = requests.get(standings_url)

In [92]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [93]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [94]:
team_urls

['https://fbref.com/en/squads/b8fd03ef/2021-2022/Manchester-City-Stats',
 'https://fbref.com/en/squads/822bd0ba/2021-2022/Liverpool-Stats',
 'https://fbref.com/en/squads/cff3d9bb/2021-2022/Chelsea-Stats',
 'https://fbref.com/en/squads/361ca564/2021-2022/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/18bb7c10/2021-2022/Arsenal-Stats',
 'https://fbref.com/en/squads/19538871/2021-2022/Manchester-United-Stats',
 'https://fbref.com/en/squads/7c21e445/2021-2022/West-Ham-United-Stats',
 'https://fbref.com/en/squads/a2d435b3/2021-2022/Leicester-City-Stats',
 'https://fbref.com/en/squads/d07537b9/2021-2022/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8cec06e1/2021-2022/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/b2b47a98/2021-2022/Newcastle-United-Stats',
 'https://fbref.com/en/squads/47c64c55/2021-2022/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/cd051869/2021-2022/Brentford-Stats',
 'https://fbref.com/en/squads/8602292d/2021-2022/Aston-Vill

In [95]:
data = requests.get(team_urls[0])

# First main Table Scores & Fixtures 1

In [96]:
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [97]:
matches.shape

(58, 19)

# Second Table/Adding attributes to first one 2

Table shooting need to merge attributes like:
- date
- Sh
- Dist
- FK
- PK
- PKatt

Need to think aboud expected column

In [98]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [99]:
data = requests.get(f"https://fbref.com{links[0]}")

In [100]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [101]:
shooting.shape

(59, 26)

In [102]:
shooting.columns = shooting.columns.droplevel()

In [103]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [104]:
team_data.shape

(58, 25)

# Third Table/Adding attributes to team_data one 3

Table Goalkeeping need to merge attributes like:
- date
- SoTA
- PKA
- PKsv
- PKm

Need to think about expected column

In [105]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/keeper' in l]

In [106]:
data = requests.get(f"https://fbref.com{links[0]}")

In [107]:
keeper = pd.read_html(data.text, match="Goalkeeping")[0]

In [108]:
keeper.shape

(59, 37)

In [109]:
keeper.columns = keeper.columns.droplevel()

In [110]:
#list(keeper.columns)

In [111]:
team_data = team_data.merge(keeper[["Date", "SoTA", "PKA", "PKsv", "PKm"]], on="Date")

In [112]:
team_data.shape

(58, 29)

# Next table Passing 4

Table Passing need to merge attributes like:
- date
- Cmp_Total
- Att_Total
- TotDist
- PrgDist

Need to think about expected column

In [113]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/passing' in l]

In [114]:
data = requests.get(f"https://fbref.com{links[0]}")

In [115]:
passing = pd.read_html(data.text, match="Passing")[0]

In [116]:
passing.shape

(59, 33)

In [117]:
passing.columns = passing.columns.droplevel()

In [118]:
passing.shape

(59, 33)

In [119]:
passing.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Cmp', 'Att', 'Cmp%', 'TotDist', 'PrgDist', 'Cmp', 'Att',
       'Cmp%', 'Cmp', 'Att', 'Cmp%', 'Cmp', 'Att', 'Cmp%', 'Ast', 'xAG', 'xA',
       'KP', '1/3', 'PPA', 'CrsPA', 'Prog', 'Match Report'],
      dtype='object')

In [120]:
passing.columns.values[10] = "Cmp_Total"
passing.columns.values[11] = "Att_Total"

In [121]:
passing.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Cmp_Total', 'Att_Total', 'Cmp%', 'TotDist', 'PrgDist',
       'Cmp', 'Att', 'Cmp%', 'Cmp', 'Att', 'Cmp%', 'Cmp', 'Att', 'Cmp%', 'Ast',
       'xAG', 'xA', 'KP', '1/3', 'PPA', 'CrsPA', 'Prog', 'Match Report'],
      dtype='object')

In [122]:
team_data = team_data.merge(passing[["Date", "Cmp_Total", "Att_Total", "TotDist", "PrgDist"]], on="Date")

In [123]:
team_data.shape

(58, 33)

In [124]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,PK,PKatt,SoTA,PKA,PKsv,PKm,Cmp_Total,Att_Total,TotDist,PrgDist
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,...,0,0,4,1,0,0,NaN,NaN,NaN,NaN
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,0,0,2,0,0,0,493.0,602.0,7528.0,2608.0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,0,0,0,0,0,0,709.0,781.0,11832.0,3037.0
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,0,0,0,0,0,0,721.0,813.0,10729.0,3207.0
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,0,0,1,0,0,0,535.0,618.0,9246.0,2784.0


# Next table Pass Types 5

Table Passi Types to merge attributes like:
- TI
- CK

Need to think about expected column

In [125]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/passing_types' in l]

In [126]:
data = requests.get(f"https://fbref.com{links[0]}")

In [127]:
pass_types = pd.read_html(data.text, match="Pass Types")[0]

In [128]:
pass_types.shape

(59, 26)

In [129]:
pass_types.columns = pass_types.columns.droplevel()

In [130]:
pass_types.shape

(59, 26)

In [131]:
team_data = team_data.merge(pass_types[["Date", "TI", "CK"]], on="Date")

In [132]:
team_data.shape

(58, 35)

# Next table Goal and Shot Creation 6

Table Goal and Shot need to merge attributes like:
- SCA
- GCA

Need to think about expected column

In [133]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/gca' in l]

In [134]:
data = requests.get(f"https://fbref.com{links[0]}")

In [135]:
gca = pd.read_html(data.text, match="Goal and Shot Creation")[0]

In [136]:
gca.shape

(59, 25)

In [137]:
gca.columns = gca.columns.droplevel()

In [138]:
gca.shape

(59, 25)

In [139]:
team_data = team_data.merge(gca[["Date", "SCA", "GCA"]], on="Date")

In [140]:
team_data.shape

(58, 37)

# Next table Defensive Actions 7

Table Defensive need to merge attributes like:
- Tkl_Defensive
- TklW_Defensive
- Blocks_Defensive

Need to think about expected column

In [141]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/defense' in l]

In [142]:
data = requests.get(f"https://fbref.com{links[0]}")

In [143]:
defension_actions = pd.read_html(data.text, match="Defensive Actions")[0]

In [144]:
defension_actions.shape

(59, 27)

In [145]:
defension_actions.columns = defension_actions.columns.droplevel()

In [146]:
defension_actions.shape

(59, 27)

In [147]:
defension_actions.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Tkl', 'TklW', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Tkl',
       'Att', 'Tkl%', 'Past', 'Blocks', 'Sh', 'Pass', 'Int', 'Tkl+Int', 'Clr',
       'Err', 'Match Report'],
      dtype='object')

In [148]:
defension_actions.columns.values[10] = "Tkl_Defensive"
defension_actions.columns.values[11] = "TklW_Defensive"
defension_actions.columns.values[19] = "Blocks_Defensive"

In [149]:
defension_actions.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Tkl_Defensive', 'TklW_Defensive', 'Def 3rd', 'Mid 3rd',
       'Att 3rd', 'Tkl', 'Att', 'Tkl%', 'Past', 'Blocks_Defensive', 'Sh',
       'Pass', 'Int', 'Tkl+Int', 'Clr', 'Err', 'Match Report'],
      dtype='object')

In [150]:
team_data = team_data.merge(defension_actions[["Date", "Tkl_Defensive", "TklW_Defensive", "Blocks_Defensive"]], on="Date")
team_data.shape

(58, 40)

# Next table Possesion 8

Table Possesion need to merge attributes like:
- date
- Touches
- Def Pen
- Def 3rd
- Mid 3rd
- Att 3rd
- Dribbles_Succ
- Dribbles_Att
- Rec

Need to think about expected column

In [151]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/possession' in l]

In [152]:
data = requests.get(f"https://fbref.com{links[0]}")

In [153]:
possession = pd.read_html(data.text, match="Possession")[0]

In [154]:
possession.shape

(59, 26)

In [155]:
possession.columns = possession.columns.droplevel()

In [156]:
possession.shape

(59, 26)

In [157]:
possession.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Poss', 'Touches', 'Def Pen', 'Def 3rd', 'Mid 3rd',
       'Att 3rd', 'Att Pen', 'Live', 'Succ', 'Att', 'Succ%', 'Mis', 'Dis',
       'Rec', 'Prog', 'Match Report'],
      dtype='object')

In [158]:
possession.columns.values[18] = "Dribbles_Succ"
possession.columns.values[19] = "Dribles_Att"

In [159]:
possession.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Poss', 'Touches', 'Def Pen', 'Def 3rd', 'Mid 3rd',
       'Att 3rd', 'Att Pen', 'Live', 'Dribbles_Succ', 'Dribles_Att', 'Succ%',
       'Mis', 'Dis', 'Rec', 'Prog', 'Match Report'],
      dtype='object')

In [160]:
team_data = team_data.merge(possession[["Date", "Touches", "Def Pen", "Def 3rd", "Mid 3rd", "Att 3rd", "Dribbles_Succ", "Dribles_Att", "Rec"]], on="Date")
team_data.shape

(58, 48)

# Next table Miscellaneous stats 9

Table Possesion need to merge attributes like:
- date
- CrdY
- CrdR
- 2CrdY
- Fls
- Fld
- Off
- PKwon
- PKcon
- OG

Need to think about expected column

In [161]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/misc' in l]

In [162]:
data = requests.get(f"https://fbref.com{links[0]}")

In [163]:
msc = pd.read_html(data.text, match="Miscellaneous Stats")[0]

In [164]:
msc.shape

(59, 27)

In [165]:
msc.columns = msc.columns.droplevel()

In [166]:
msc.shape

(59, 27)

In [167]:
team_data = team_data.merge(msc[["Date", "CrdY", "CrdR", "2CrdY", "Fls", "Fld", "Off", "PKwon", "PKcon", "OG"]], on="Date")
team_data.shape

(58, 57)

# Convert to CSV file

In [170]:
team_data.to_csv("EPL_2021.csv", index=0)